In [ ]:
#Carbon emission  LCOE
import matplotlib.pyplot as plt
import numpy as np


CO2_per_kWh = 0.4  # kg CO2/kWh, 可按当地电网碳排放因子调整
annual_CO2_reduction = annual_energy * CO2_per_kWh
cum_CO2_reduction = np.cumsum(annual_CO2_reduction)  # 累计 CO2 减排
total_CO2_reduction = cum_CO2_reduction[-1]  # 30年累计 CO2 减排总量


fig, ax1 = plt.subplots(figsize=(10,5))

# Right axis: Annual discounted cost, annual energy and carbon reduction
ax2 = ax1.twinx()
ax2.bar(years_range - 0.15, annual_cost * discount_factors, width=0.3, color='orange', alpha=0.7, label='Discounted Annual Cost')
ax2.bar(years_range + 0.15, annual_energy, width=0.3, color='#B19CCB', alpha=0.7, label='Annual Energy (kWh)')
ax2.plot(years_range, annual_CO2_reduction, color='#4198AC', marker='s', linestyle='-', label='Annual CO₂ Reduction (kg)')
ax2.plot(years_range, cum_CO2_reduction, color='green', linestyle='--', label=f'Cumulative CO₂ Reduction (Total: {total_CO2_reduction:.0f} kg)')
ax2.set_ylabel('Annual Cost (USD) / Energy (kWh) / CO₂ (kg)', color='#7570b3')
ax2.tick_params(axis='y', labelcolor='#7570b3')

# Left axis: Cumulative discounted LCOE
ax1.plot(years_range, LCOE_trend, marker='o', color='red', label='Cumulative Discounted LCOE')
ax1.axhline(y=electricity_price, color='red', linestyle='--', label=f'Grid Price ({electricity_price:.2f} USD/kWh)')
ax1.set_xlabel('Year')
ax1.set_ylabel('LCOE (USD/kWh)', color='#6656A6')
ax1.tick_params(axis='y', labelcolor='#6656A6')
ax1.set_title('30-Year System LCOE, Cost, Energy and CO₂ Reduction')
ax1.grid(True, linestyle='--', alpha=0.5)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# -------------------- data --------------------
construction_file = r'D:\1015 model DB\NewYorkconstruction_summary.csv'
epw_file = r'D:\1015 model DB\USA_NY_New.York-John.F.Kennedy.Intl.AP.744860_TMY3.epw'

# -------------------- parameter --------------------
classrooms = [
    'Block 3 - Mult_Class_1_Pod_3_ZN_1_FLR_2 Space',
    'Block 3 - Mult_Class_1_Pod_3_ZN_1_FLR_3 Space',
    'Block 4 - Mult_Class_1_Pod_1_ZN_1_FLR_4 Space'
]

met = 1.2
clo = 0.5
v_air = 0.1  # m/s Default wind speed
solar_gain_factor = 0.5  # Proportion of solar radiation transmitted through the window
internal_gain = 50       # W/m² Assumed internal heat sources (students + equipment)

# -------------------- construction informantion --------------------
df_constr = pd.read_csv(construction_file)

# -------------------- read EPW files --------------------
def read_epw(epw_file):
    epw_data = []
    with open(epw_file, 'r') as f:
        lines = f.readlines()
        for line in lines[8:]:  # 前8行是头信息
            parts = line.strip().split(',')
            Ta = float(parts[6])   # 干球温度
            RH = float(parts[8])   # 相对湿度
            GHI = float(parts[14]) # 全球水平辐射
            epw_data.append({'Ta': Ta, 'RH': RH, 'GHI': GHI})
    return pd.DataFrame(epw_data)

df_epw = read_epw(epw_file)

# -------------------- temperature depict --------------------
def estimate_indoor_temp(zone_name, df_constr, df_epw, pemfc_effect=0):
    zone = df_constr[df_constr['Zone'] == zone_name]
    U_total = 0
    area_total = 0
    window_solar_area = 0
    for idx, row in zone.iterrows():
        U = row['U-Value (W/K-m2)']
        A = row['Area-Nett (m2)']
        U_total += U * A
        area_total += A
        if row['Element'].lower() == 'window':
            window_solar_area += A
    U_total = max(U_total, 0.01)  # 避免除零
    T_indoor = df_epw['Ta'].values + (
        internal_gain * area_total + 
        df_epw['GHI'].values * window_solar_area * solar_gain_factor + 
        pemfc_effect
    ) / U_total
    RH = df_epw['RH'].values
    return pd.DataFrame({
        'Air Temperature (C)': T_indoor,
        'Mean Radiant Temperature (C)': T_indoor,
        'Relative Humidity (%)': RH
    })

# -------------------- PMV/PPD  --------------------
def ppd_from_pmv(pmv_value):
    return 100 - 95 * np.exp(-0.03353 * pmv_value**4 - 0.2179 * pmv_value**2)

def calc_pmv_ppd(Ta, Tr, RH, v_air, met=1.2, clo=0.5):
    pa = RH / 100 * 10 * np.exp(16.6536 - 4030.183 / (Ta + 235))
    M = met * 58.15  # W/m²
    W = 0
    Icl = clo * 0.155
    fcl = 1.05 + 0.1 * Icl
    Tcl = Ta + (35.5 - Ta) / (3.5 * Icl + 0.1)
    PMV = (0.303 * np.exp(-0.036 * M) + 0.028) * (
        (M - W)
        - 3.05e-3 * (5733 - 6.99 * M - pa)
        - 0.42 * ((M - W) - 58.15)
        - 1.7e-5 * M * (5867 - pa)
        - 0.0014 * M * (34 - Ta)
        - 3.96e-8 * fcl * ((Tcl + 273)**4 - (Tr + 273)**4)
        - fcl * 3.05e-3 * (5733 - 6.99 * M - pa)
    )
    PPD = ppd_from_pmv(PMV)
    return PMV, PPD

# -------------------- classrooms --------------------
results = {}
extreme_stats = {}
for cls in classrooms:
    df_before = estimate_indoor_temp(cls, df_constr, df_epw, pemfc_effect=0)
    df_after  = estimate_indoor_temp(cls, df_constr, df_epw, pemfc_effect=2000)
    PMV_before, PPD_before = [], []
    PMV_after, PPD_after = [], []
    for i in range(len(df_before)):
        pmv, ppd = calc_pmv_ppd(df_before.iloc[i, 0], df_before.iloc[i, 1], df_before.iloc[i, 2], v_air, met, clo)
        PMV_before.append(pmv)
        PPD_before.append(ppd)
        pmv, ppd = calc_pmv_ppd(df_after.iloc[i, 0], df_after.iloc[i, 1], df_after.iloc[i, 2], v_air, met, clo)
        PMV_after.append(pmv)
        PPD_after.append(ppd)
    df_before['PMV'] = PMV_before
    df_before['PPD'] = PPD_before
    df_after['PMV'] = PMV_after
    df_after['PPD'] = PPD_after
    results[cls] = {'before': df_before, 'after': df_after}
    
    extreme_before = np.mean((df_before['PMV'] < -3) | (df_before['PMV'] > 3))
    extreme_after  = np.mean((df_after['PMV'] < -3) | (df_after['PMV'] > 3))
    extreme_stats[cls] = {'before': extreme_before, 'after': extreme_after}

# -------------------- Visualization --------------------
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 10))
colors_before = ['#539DCC', '#E595A4', '#CE4459']
colors_after  = ['#448870', '#6656A6', '#EA9E58']
scale_factors = [0.015, 1/60, 0.1] 


titles = [
    "Classroom1 (North)",
    "Classroom2 (South)",
    "Classroom3 (East)"
]

for i, cls in enumerate(classrooms, 1):
    plt.subplot(3, 1, i)
    before_scaled = results[cls]['before']['PMV'] * scale_factors[i-1]
    after_scaled  = results[cls]['after']['PMV']  * scale_factors[i-1]

    plt.plot(before_scaled, label='Before PEMFC', 
             color=colors_before[i-1], linewidth=1.0, linestyle='--')
    plt.plot(after_scaled, label='After PEMFC', 
             color=colors_after[i-1], linewidth=1.0, linestyle='-')

    
    plt.title(titles[i-1], fontsize=12)
    plt.ylabel('PMV ')
    plt.xlabel('Hour')
    plt.legend(fontsize=9)
    plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ===================== data path =====================
doe_file = r"D:\1015 model DB\DOE_static_estimate.csv"

# ===================== read data =====================
df = pd.read_csv(doe_file)
df.columns = df.columns.str.strip()

# ===================== Calculation error=====================
#Tenp
T_opt_min, T_opt_max = 21, 23
df['ΔT'] = np.where(df['Operative Temperature'] < T_opt_min,
                    T_opt_min - df['Operative Temperature'],
                    np.where(df['Operative Temperature'] > T_opt_max,
                             df['Operative Temperature'] - T_opt_max, 0))

# humidity
RH_opt_min, RH_opt_max = 30, 60
df['ΔRH'] = np.where(df['outdoorRH'] < RH_opt_min,
                     (RH_opt_min - df['outdoorRH'])/10,
                     np.where(df['outdoorRH'] > RH_opt_max,
                              (df['outdoorRH'] - RH_opt_max)/10, 0))

# lighting
GHI_opt_min, GHI_opt_max = 100, 1000
df['ΔGHI'] = np.where(df['Global horizontal irradiance'] < GHI_opt_min,
                      (GHI_opt_min - df['Global horizontal irradiance'])/100,
                      np.where(df['Global horizontal irradiance'] > GHI_opt_max,
                               (df['Global horizontal irradiance'] - GHI_opt_max)/100, 0))

# wind speed
Wind_opt_min, Wind_opt_max = 0.1, 0.8
df['ΔWind'] = np.where(df['Wind Speed'] < Wind_opt_min,
                       (Wind_opt_min - df['Wind Speed'])*5,
                       np.where(df['Wind Speed'] > Wind_opt_max,
                                (df['Wind Speed'] - Wind_opt_max)*5, 0))

a_T, a_RH, a_GHI, a_Wind = 1.5, 1.0, 0.5, 1.0

# =====================Learning performance without PEMFC =====================
df['LPI_noPEMFC'] = 100 - (a_T * df['ΔT'] + a_RH * df['ΔRH'] + a_GHI * df['ΔGHI'] + a_Wind * df['ΔWind'])
df['LPI_noPEMFC'] = df['LPI_noPEMFC'].clip(lower=0, upper=100)

# ===================== AFTER PEMFC =====================
df['ΔT_PEMFC'] = df['ΔT'] * 0.75

df['PEMFC_boost'] = np.where((df['Global horizontal irradiance'] < 200) & (df['Wind Speed'] < 0.2), 5, 0)

# RECACULATE LPI
df['LPI_PEMFC'] = 100 - (a_T * df['ΔT_PEMFC'] + a_RH * df['ΔRH'] + a_GHI * df['ΔGHI'] + a_Wind * df['ΔWind']) + df['PEMFC_boost']
df['LPI_PEMFC'] = df['LPI_PEMFC'].clip(lower=0, upper=100)

# ===================== timestamp =====================
if 'Timestamp' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)

# ===================== pictures =====================
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['LPI_noPEMFC'], label='Without PEMFC', color='#FF8F9F', alpha=0.9)
plt.plot(df.index, df['LPI_PEMFC'], label='With PEMFC', color='#88CEBF', linewidth=0.6)
plt.axhline(85, color='#6656A6', linestyle='--', label='Good Performance Threshold')
plt.axhline(70, color='black', linestyle='--', label='Poor Performance Threshold')
plt.title("Learning Performance Index (LPI) Before and After Adding PEMFC")
plt.xlabel("Time")
plt.ylabel("LPI (score 0-100)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ===================== output results=====================
print("=== Comparison of Learning Performance Index Statistics ===")
print("Average LPI (without PEMFC) =", round(df['LPI_noPEMFC'].mean(), 2))
print("Average LPI (with PEMFC) =", round(df['LPI_PEMFC'].mean(), 2))
print("Increase range =", round(df['LPI_PEMFC'].mean() - df['LPI_noPEMFC'].mean(), 2), "分")
print("LPI ≥ 85（without PEMFC） =", round((df['LPI_noPEMFC'] >= 85).mean()*100, 2), "%")
print("LPI ≥ 85（with PEMFC） =", round((df['LPI_PEMFC'] >= 85).mean()*100, 2), "%")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ===================== data path =====================
doe_file = r"D:\1015 model DB\DOE_static_estimate.csv"

# ===================== read data =====================
df = pd.read_csv(doe_file)
df.columns = df.columns.str.strip()

T_opt_min, T_opt_max = 21, 23
RH_opt_min, RH_opt_max = 30, 60
GHI_opt_min, GHI_opt_max = 100, 1000
Wind_opt_min, Wind_opt_max = 0.1, 0.8


a_T, a_RH, a_GHI, a_Wind = 1.5, 1.0, 0.5, 1.0

# ===================== classrooms=====================
n = len(df)
df_classrooms = [
    df.iloc[:n//3].copy(),        # Classroom1
    df.iloc[n//3:2*n//3].copy(),  # Classroom2
    df.iloc[2*n//3:].copy()       # Classroom3
]
titles = ["Classroom 1", "Classroom 2", "Classroom 3"]

# ===================== LPI =====================
def calc_LPI(df):
    df = df.copy()
    df['ΔT'] = np.where(df['Operative Temperature'] < T_opt_min,
                        T_opt_min - df['Operative Temperature'],
                        np.where(df['Operative Temperature'] > T_opt_max,
                                 df['Operative Temperature'] - T_opt_max, 0))
    df['ΔRH'] = np.where(df['outdoorRH'] < RH_opt_min,
                         (RH_opt_min - df['outdoorRH'])/10,
                         np.where(df['outdoorRH'] > RH_opt_max,
                                  (df['outdoorRH'] - RH_opt_max)/10, 0))
    df['ΔGHI'] = np.where(df['Global horizontal irradiance'] < GHI_opt_min,
                          (GHI_opt_min - df['Global horizontal irradiance'])/100,
                          np.where(df['Global horizontal irradiance'] > GHI_opt_max,
                                   (df['Global horizontal irradiance'] - GHI_opt_max)/100, 0))
    df['ΔWind'] = np.where(df['Wind Speed'] < Wind_opt_min,
                           (Wind_opt_min - df['Wind Speed'])*5,
                           np.where(df['Wind Speed'] > Wind_opt_max,
                                    (df['Wind Speed'] - Wind_opt_max)*5, 0))
    df['LPI'] = 100 - (a_T * df['ΔT'] + a_RH * df['ΔRH'] + a_GHI * df['ΔGHI'] + a_Wind * df['ΔWind'])
    df['LPI'] = df['LPI'].clip(lower=0, upper=100)
    return df

# ===================== PEMFC Adjustment Function =====================
def apply_PEMFC(df, gain_factor=0.75, boost_value=5):
    df_after = df.copy()
    # 温度调节（将温度偏差减少）
    df_after['ΔT_PEMFC'] = df_after['ΔT'] * gain_factor
    # 小幅提升光照/风速条件下的 LPI
    df_after['LPI_PEMFC'] = 100 - (a_T * df_after['ΔT_PEMFC'] + a_RH * df_after['ΔRH'] +
                                   a_GHI * df_after['ΔGHI'] + a_Wind * df_after['ΔWind']) + boost_value
    df_after['LPI_PEMFC'] = df_after['LPI_PEMFC'].clip(0, 100)
    return df_after

# ===================== classrooms=====================
results = []
for df_zone, title in zip(df_classrooms, titles):
    # 时间索引
    if 'Timestamp' in df_zone.columns:
        df_zone['Timestamp'] = pd.to_datetime(df_zone['Timestamp'])
        df_zone.set_index('Timestamp', inplace=True)
    
    df_before = calc_LPI(df_zone)
    df_after = apply_PEMFC(df_before)
    
    results.append({'before': df_before, 'after': df_after, 'title': title})

# ===================== Visualization =====================
plt.figure(figsize=(12, 10))
for i, data in enumerate(results, 1):
    plt.subplot(3, 1, i)
    plt.plot(data['before'].index, data['before']['LPI'], label='Before PEMFC', color='#FF8F9F', alpha=0.6, linewidth=1.0)
    plt.plot(data['after'].index, data['after']['LPI_PEMFC'], label='After PEMFC', color='#88CEBF', alpha=1, linewidth=1.0)
    plt.axhline(85, color='#6656A6', linestyle='--', label='Good Threshold' if i==1 else "")
    plt.axhline(70, color='black', linestyle='--', label='Poor Threshold' if i==1 else "")
    plt.title(data['title'], fontsize=12)
    plt.ylabel("LPI (score 0-100)")
    plt.legend(fontsize=9)
    plt.grid(alpha=0.3)
plt.xlabel("Time")
plt.tight_layout()
plt.show()

# ===================== results =====================
print("========= Statistics on Improvement in Learning Performance（By Classroom） =========")
for data in results:
    mean_before = data['before']['LPI'].mean()
    mean_after = data['after']['LPI_PEMFC'].mean()
    good_before = (data['before']['LPI'] >= 85).mean() * 100
    good_after = (data['after']['LPI_PEMFC'] >= 85).mean() * 100
    print(f"{data['title']}:")
    print(f"   Averange LPI: Before = {mean_before:.2f}, After = {mean_after:.2f}, 提升 = {mean_after - mean_before:.2f}")
    print(f"  Proportion of time with LPI ≥ 85: Before = {good_before:.2f}%, After = {good_after:.2f}%\n")